# Assiment 10

#### Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 43.5 MB/s eta 0:00:00


In [3]:
# requirementss
%pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
%pip install git+https://github.com/jgrss/geowombat  -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 11.2 MB/s eta 0:00:00


In [4]:
%pip install rasterstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.5 MB/s eta 0:00:00


In [5]:
!pip install geopandas

In [1]:
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd
import geowombat as gw

import rasterio
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import numpy as np
import chardet

import pyproj
from shapely.ops import transform

from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

In [2]:
shapefile_path = '/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp'
department = gpd.read_file(shapefile_path)
print(department.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [3]:
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Define a function to apply the transformation
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Apply the transformation to the geometries
department['geometry'] = department['geometry'].apply(apply_transform)

In [4]:
print(department.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [10]:
# List of raster file paths
tif_file_path = [
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif'
]

# Ciclo para cargar cada archivo raster
for file_path in tif_file_path:
    # Abrir el archivo raster usando geowombat.open
    with gw.open(file_path) as src:
        # Visualizar información del raster
        data = src.where(src != 0)

data.head(3)

<xarray.DataArray (band: 1, y: 3, x: 3)>
dask.array<getitem, shape=(1, 3, 3), dtype=float32, chunksize=(1, 3, 3), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * x        (x) float64 -7.041e+06 -7.041e+06 -7.041e+06
  * y        (y) float64 1e+06 1e+06 1e+06
Attributes: (12/13)
    transform:           (10.0, 0.0, -7041000.0, 0.0, -10.0, 1000000.0)
    crs:                 +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +unit...
    res:                 (10.0, 10.0)
    is_tiled:            1
    nodatavals:          (255.0,)
    _FillValue:          255.0
    ...                  ...
    offsets:             (0.0,)
    AREA_OR_POINT:       Area
    filename:            /content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLO...
    resampling:          nearest
    _data_are_separate:  0
    _data_are_stacked:   0

In [4]:
from rasterio.windows import from_bounds

# Ensure the shapefile is in the same CRS as the raster
raster_path = '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif'
with rasterio.open(raster_path) as src:

    # Create a mask for the intersection area
    shapes = [feature["geometry"] for _, feature in department.iterrows()]
    out_image, out_transform = mask(src, shapes, crop=True)
    out_meta = src.meta.copy()

# Update the metadata for the new (clipped) raster
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped raster to a new file
output_raster_path = '/content/sample_data/raster_1.tif'
with rasterio.open(output_raster_path, 'w', **out_meta) as dst:
    dst.write(out_image)

In [ ]:
with gw.config.update(sensor='bgr'):
    with gw.open('/content/sample_data/raster_1.tif') as src:
        df = src.gw.extract(
            '/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp',
            band_names=src.band.values.tolist()
        )
print(df)

03:31:05:WARNING:336:xarray_.warp_open:  The new bands, ['blue', 'green', 'red'], do not match the sensor bands, [1].


In [ ]:
import geowombat as gw

from geowombat.data import l8_224078_20200518, l8_224078_20200518_points

with gw.open(l8_224078_20200518) as src:
    df = src.gw.extract(l8_224078_20200518_points)

In [2]:
import geowombat as gw
import geopandas as gpd

polys = gpd.read_file('/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp')

with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif') as src:
    print(src)
    clipped = src.gw.clip_by_polygon(
        polys,
        mask_data=True,
        expand_by=1
    )
    print(clipped)

<xarray.DataArray (band: 1, y: 100000, x: 100000)>
dask.array<open_rasterio-0519ba889d340fce4bf86df6db4645f3<this-array>, shape=(1, 100000, 100000), dtype=uint8, chunksize=(1, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * x        (x) float64 -8.041e+06 -8.041e+06 ... -7.041e+06 -7.041e+06
  * y        (y) float64 -5.0 -15.0 -25.0 -35.0 ... -1e+06 -1e+06 -1e+06 -1e+06
Attributes: (12/13)
    transform:           (10.0, 0.0, -8041000.0, 0.0, -10.0, 0.0)
    crs:                 +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +unit...
    res:                 (10.0, 10.0)
    is_tiled:            1
    nodatavals:          (255.0,)
    _FillValue:          255.0
    ...                  ...
    offsets:             (0.0,)
    AREA_OR_POINT:       Area
    filename:            /content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLO...
    resampling:          nearest
    _data_are_separate:  0
    _data_are_stacked:   0
<xarray.DataArray (band: 1, y: 99524, 

In [23]:
import geowombat as gw
import geopandas as gpd

# Leer el shapefile de polígonos
polys = gpd.read_file('/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp')

# Abrir el raster Landsat
with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif') as src:
    # Recortar el raster por los polígonos
    clipped = src.gw.clip_by_polygon(
        polys,
        mask_data=True,
        expand_by=1
    )

    # Ajustar el tamaño de los bloques del dataset para que sea múltiplo de 16
    clipped = clipped.coarsen(x=16, y=16, boundary='pad').mean()

    # Guardar el resultado como un archivo GeoTIFF
    gw.to_raster(clipped, '/content/sample_data/raster_2.tif', dtype=src.dtype)

RasterBlockError: The height and width of dataset blocks must be multiples of 16

In [7]:
# Lista de rutas de archivos raster
tif_file_path = [
    '/content/sample_data/raster_1.tif'
]

In [ ]:
# We generate a list to obtain the raster data
raster_data = []

# Iterate over each raster file
for raster_path in tif_file_path:
    # We intercept the raster information with the shp file
    stats = zonal_stats(department, raster_path, stats="count sum", categorical=True, all_touched=True)
    # Convert Statistics to DataFrame
    stats_df = pd.DataFrame(stats)
    # In the created list we concatenate the geometry and raster information of the departments
    df = pd.concat([department.reset_index(drop=True), stats_df], axis=1)
    raster_data.append(df)

In [ ]:
# Concatenar todos los DataFrames en all_stats en un único DataFrame
final_df_raster = pd.concat(raster_data, ignore_index=True)

# Verificar el resultado
print(final_df_raster.head())